In [114]:
import pandas as pd
import numpy as np
import glob
import tqdm
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/switrs.sqlite")
q = """
SELECT collision_date, longitude, latitude FROM collisions
WHERE collision_date IS NOT NULL
AND longitude IS NOT NULL
AND latitude IS NOT NULL    
AND collision_date <= '2017-12-31'  -- 2018 is incomplete
AND collision_date >= '2017-01-01'
"""
df = pd.read_sql_query(q, con)

# Round the coordinates, given that NASA only looks at grids of 0.5 x 0.5
df.latitude = round(df.latitude*2)/2
df.longitude = round(df.longitude*2)/2
df.orig = df.copy()

# Initially dropped duplicates because that speeds up our investigation
df.drop_duplicates(inplace=True)
df.columns = ['collision_date', 'LON', 'LAT']

# Load in the data retrieved from NASA (which are seperate CSV files made up of 4.5 x 4.5 degree Region data)
# See seperate Notebook for loading NASA data
path = 'data' # use your path
all_files = glob.glob(path + "/POWER*.csv")

li = []

for filename in all_files:
    df_ap = pd.read_csv(filename, index_col=None, header=10)
    li.append(df_ap)

frame = pd.concat(li, axis=0, ignore_index=True)
frame['DATE'] = pd.to_datetime(frame['DOY'], format='%j').dt.strftime('2017-%m-%d')
frame.drop(['YEAR', 'DOY'], axis=1, inplace=True)

In [108]:
# Create functions for finding the closest point to a point and matching another field to it
from scipy.spatial.distance import cdist
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2, y, col3):
    """ Match value x from col1 row to value in col2. """
    return df[(df[col1] == x) & (df[col3] == y)][col2].values[0]


df1 = frame
df2 = df.copy()

import time

t0 = time.time()

df1['point'] = [(x, y) for x,y in zip(df1['LAT'], df1['LON'])]
df2['point'] = [(x, y) for x,y in zip(df2['LAT'], df2['LON'])]

df2['closest'] = [closest_point(x, list(df1['point'])) for x in tqdm.tqdm(df2['point'])]
df2['T2M'] = [match_value(df1, 'point', x, 'T2M', y, 'DATE') for x,y in tqdm.tqdm(zip(df2['closest'], df2['collision_date']))]

t1 = time.time()

total = t1-t0
print('Finished in ')
print(total)

In [110]:
# Write this file to a CSV for intermediate checkpoint saving
df2.to_csv('data/supermegafinal.csv')

In [ ]:
# Now, initialize the original data frame
df_orig.columns = ['collision_date', 'LON', 'LAT']

In [124]:
# Create the points (this time a three element tuple) for the original date sets
df_orig['point'] = [(x,y,z) for x,y,z in zip(df_orig['LAT'], df_orig['LON'], df_orig['collision_date'])]
df2['point_date'] = [(x,y,z) for x,y,z in zip(df2['LAT'], df2['LON'], df2['collision_date'])]

In [128]:
# Match the original dataset to the already matched data of the unique values, done earlier
def match_value_2(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[(df[col1] == x)][col2].values[0]

df_orig['T2M'] = [match_value_2(df2, 'point_date', x, 'T2M') for x in tqdm.tqdm(df_orig['point'])]

100%|██████████| 292296/292296 [15:25<00:00, 315.79it/s]


In [129]:
# Write the final file to CSV
df_orig.to_csv('data/collisionweather.csv')

In [136]:
df_orig['T2M'].value_counts().to_csv('data/collisionweather_valuecounts.csv', header=False)

In [142]:
df_orig['T2M']

0         33.74
1         14.52
2         14.89
3         16.42
4         21.98
          ...  
292291    11.86
292292    15.08
292293    15.08
292294    15.08
292295    12.76
Name: T2M, Length: 292296, dtype: float64

In [6]:
import pandas as pd
df_orig = pd.read_csv('data/collisionweather.csv')

In [7]:
df_orig

,Unnamed: 0,collision_date,LON,LAT,point,T2M
0,0,2017-08-31,-119.5,36.5,"(36.5, -119.5, '2017-08-31')",33.74
1,1,2017-02-20,-118.0,33.5,"(33.5, -118.0, '2017-02-20')",14.52
2,2,2017-04-23,-119.0,35.5,"(35.5, -119.0, '2017-04-23')",14.89
3,3,2017-05-23,-118.0,33.5,"(33.5, -118.0, '2017-05-23')",16.42
4,4,2017-07-12,-122.5,39.0,"(39.0, -122.5, '2017-07-12')",21.98
...,...,...,...,...,...,...
292291,292291,2017-04-08,-117.0,34.0,"(34.0, -117.0, '2017-04-08')",11.86
292292,292292,2017-12-25,-118.0,34.0,"(34.0, -118.0, '2017-12-25')",15.08
292293,292293,2017-12-25,-118.0,34.0,"(34.0, -118.0, '2017-12-25')",15.08
292294,292294,2017-12-25,-118.0,34.0,"(34.0, -118.0, '2017-12-25')",15.08


In [30]:
collisions_grouped = pd.DataFrame(pd.Series(df_orig.groupby('collision_date').mean()['T2M']))
collisions_grouped['count'] = pd.Series(df_orig['collision_date'].value_counts().sort_index())
collisions_grouped.columns = ['temp', 'count']
collisions_grouped.to_csv('data/collisions_grouped.csv', header=False, index=False)